In [ ]:
import os
import tensorflow as tf
from typing import Any, Dict, Text
from modules.merge_dataset import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import pipeline, components
from absl import logging
from tfx import components
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import \
    LatestBlessedModelStrategy
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [2]:
PIPELINE_NAME = "movie-recommender-pipeline"

# pipeline inputs
MOVIES_DATA_ROOT = "data/movies"
RATING_DATA_ROOT = "data/merge"
MOVIES_TRANSFORM_MODULE_FILE = "modules/movies_transform.py"
RATING_TRANSFORM_MODULE_FILE = "modules/rating_transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [3]:
context = InteractiveContext(pipeline_root=pipeline_root)

In [4]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(
            name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
    ])
)       

In [5]:
movies_example_gen = components.CsvExampleGen(
    input_base=MOVIES_DATA_ROOT,
    output_config=output
)
context.run(movies_example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 85
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [6]:
rating_example_gen = components.CsvExampleGen(
    input_base=RATING_DATA_ROOT,
    output_config=output
)
context.run(rating_example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 86
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
movies_statistics_gen = components.StatisticsGen(
    examples=movies_example_gen.outputs["examples"]
)
context.run(movies_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 87
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
rating_statistics_gen = components.StatisticsGen(
    examples=rating_example_gen.outputs["examples"]
)
context.run(rating_statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 88
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [9]:
movies_schema_gen = components.SchemaGen(
    statistics=movies_statistics_gen.outputs["statistics"]
)
context.run(movies_schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 89
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
rating_schema_gen = components.SchemaGen(
    statistics=rating_statistics_gen.outputs["statistics"]
)
context.run(rating_schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 90
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
movies_example_validator = components.ExampleValidator(
    statistics=movies_statistics_gen.outputs["statistics"],
    schema=movies_schema_gen.outputs["schema"],
)
context.run(movies_example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 91
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
rating_example_validator = components.ExampleValidator(
    statistics=rating_statistics_gen.outputs["statistics"],
    schema=rating_schema_gen.outputs["schema"],
)
context.run(rating_example_validator)


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 92
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [13]:
movies_transform = components.Transform(
    examples=movies_example_gen.outputs["examples"],
    schema=movies_schema_gen.outputs["schema"],
    module_file=os.path.abspath(MOVIES_TRANSFORM_MODULE_FILE)
)
context.run(movies_transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\93\.temp_path\tftransform_tmp\6e3908546c6849b3b17a2229381b8e53\assets


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\93\.temp_path\tftransform_tmp\6e3908546c6849b3b17a2229381b8e53\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 93
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [14]:
rating_transform = components.Transform(
    examples=rating_example_gen.outputs["examples"],
    schema=rating_schema_gen.outputs["schema"],
    module_file=os.path.abspath(RATING_TRANSFORM_MODULE_FILE)
)
context.run(rating_transform)

INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\94\.temp_path\tftransform_tmp\4d194546a81945cba0a0a080b7b11d84\assets


INFO:tensorflow:Assets written to: outputs\movie-recommender-pipeline\Transform\transform_graph\94\.temp_path\tftransform_tmp\4d194546a81945cba0a0a080b7b11d84\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 94
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [20]:
trainer = components.Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=rating_transform.outputs["transformed_examples"],
    transform_graph=rating_transform.outputs["transform_graph"],
    schema=rating_transform.outputs["post_transform_schema"],
    # hyperparameters=tuner.outputs["best_hyperparameters"],
    train_args=trainer_pb2.TrainArgs(
        splits=["train"],
        num_steps=500,
    ),
    eval_args=trainer_pb2.EvalArgs(
        splits=["eval"],
        num_steps=10,
    ),
    custom_config={
        "epochs": 5,
        "movies": movies_transform.outputs["transformed_examples"],
        "movies_schema": movies_transform.outputs["post_transform_schema"],
    }
)
context.run(trainer)


Epoch 1/5
500/500 [==============================] - 90s 175ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0300 - factorized_top_k/top_5_categorical_accuracy: 0.1660 - factorized_top_k/top_10_categorical_accuracy: 0.3080 - factorized_top_k/top_50_categorical_accuracy: 0.5000 - factorized_top_k/top_100_categorical_accuracy: 0.5000 - loss: 0.0000e+00 - regularization_loss: 0.0951 - total_loss: 0.0951 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.2000 - val_factorized_top_k/top_10_categorical_accuracy: 0.2000 - val_factorized_top_k/top_50_categorical_accuracy: 0.6000 - val_factorized_top_k/top_100_categorical_accuracy: 0.6000 - val_loss: 0.0000e+00 - val_regularization_loss: 0.0681 - val_total_loss: 0.0681
Epoch 2/5
500/500 [==============================] - 92s 184ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0440 - factorized_top_k/top_5_categorical_accuracy: 0.2100 - factorized_top_k/top_10_catego

In [ ]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolve")
context.run(model_resolver)

In [ ]:
pusher = components.Pusher(
    model=trainer.outputs["model"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                serving_model_dir, "movie-recommender"
            ),
        )
    )
)
context.run(pusher)

In [ ]:
loaded = tf.saved_model.load(pusher.outputs['pushed_model'].get()[0].uri)
scores, titles = loaded(["42"])

print(f"Recommendations: {titles[0][:3]}")